<a href="https://colab.research.google.com/github/oshbocker/snocast/blob/main/snocast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict

In [4]:
ground_measures_train = pd.read_csv('/content/drive/MyDrive/snocast/ground_measures_train_features.csv')
train_labels = pd.read_csv('/content/drive/MyDrive/snocast/train_labels.csv')
ground_measures_test = pd.read_csv('/content/drive/MyDrive/snocast/ground_measures_test_features.csv')
ground_measures_metadata = pd.read_csv('/content/drive/MyDrive/snocast/ground_measures_metadata.csv')
submission_format = pd.read_csv('/content/drive/MyDrive/snocast/submission_format.csv')

In [5]:
print(submission_format.shape)
submission_format.head()

(9066, 58)


,cell_id,2020-01-07,2020-01-14,2020-01-21,2020-01-28,2020-02-04,2020-02-11,2020-02-18,2020-02-25,2020-03-03,2020-03-10,2020-03-17,2020-03-24,2020-03-31,2020-04-07,2020-04-14,2020-04-21,2020-04-28,2020-05-05,2020-05-12,2020-05-19,2020-05-26,2020-06-02,2020-06-09,2020-06-16,2020-06-23,2020-06-30,2020-12-01,2020-12-08,2020-12-15,2020-12-22,2020-12-29,2021-01-05,2021-01-12,2021-01-19,2021-01-26,2021-02-02,2021-02-09,2021-02-16,2021-02-23,2021-03-02,2021-03-09,2021-03-16,2021-03-23,2021-03-30,2021-04-06,2021-04-13,2021-04-20,2021-04-27,2021-05-04,2021-05-11,2021-05-18,2021-05-25,2021-06-01,2021-06-08,2021-06-15,2021-06-22,2021-06-29
0,000863e7-21e6-477d-b799-f5675c348627,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000ba8d9-d6d5-48da-84a2-1fa54951fae1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00146204-d4e9-4cd8-8f86-d1ef133c5b6d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00211c19-7ea8-4f21-a2de-1d6216186a96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00226e82-e747-4f03-9c5d-3eef8ebe515e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
print(ground_measures_train.shape)
ground_measures_train.head()

(700, 214)


,Unnamed: 0,2013-01-01,2013-01-08,2013-01-15,2013-01-22,2013-01-29,2013-02-05,2013-02-12,2013-02-19,2013-02-26,2013-03-05,2013-03-12,2013-03-19,2013-03-26,2013-04-02,2013-04-09,2013-04-16,2013-04-23,2013-04-30,2013-05-07,2013-05-14,2013-05-21,2013-05-28,2013-06-04,2013-06-11,2013-06-18,2013-06-25,2013-12-03,2013-12-10,2013-12-17,2013-12-24,2013-12-31,2014-01-07,2014-01-14,2014-01-21,2014-01-28,2014-02-04,2014-02-11,2014-02-18,2014-02-25,...,2018-05-29,2018-06-05,2018-06-12,2018-06-19,2018-06-26,2018-12-04,2018-12-11,2018-12-18,2018-12-25,2019-01-01,2019-01-08,2019-01-15,2019-01-22,2019-01-29,2019-02-05,2019-02-12,2019-02-19,2019-02-26,2019-03-05,2019-03-12,2019-03-19,2019-03-26,2019-04-02,2019-04-09,2019-04-16,2019-04-23,2019-04-30,2019-05-07,2019-05-14,2019-05-21,2019-05-28,2019-06-04,2019-06-11,2019-06-18,2019-06-25,2019-12-03,2019-12-10,2019-12-17,2019-12-24,2019-12-31
0,CDEC:ADM,5.90,5.90,6.50,6.50,7.40,7.60,7.40,8.00,8.00,8.00,8.50,6.20,4.10,1.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.20,0.40,0.50,0.50,0.50,0.80,0.70,0.20,0.10,1.60,1.00,1.50,...,0.00,0.00,0.00,0.00,0.00,2.20,2.50,3.10,4.90,4.60,6.30,6.50,10.20,10.90,12.00,14.50,17.00,18.60,20.50,22.60,22.1,21.70,21.30,18.20,15.80,9.80,0.40,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,0.70,1.20,3.40,3.70,3.40
1,CDEC:AGP,17.52,17.54,17.85,17.39,18.03,17.70,17.65,16.66,17.21,16.26,18.00,18.11,17.96,17.94,11.41,5.39,3.82,3.60,1.06,0.74,0.74,0.67,0.35,0.42,0.13,0.01,1.72,4.16,3.99,3.88,5.70,6.44,7.36,7.24,7.72,8.24,9.49,9.29,8.75,...,1.82,1.98,1.93,1.66,1.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.60,0.20,NaN,NaN
2,CDEC:ALP,12.75,13.32,14.26,14.02,13.39,13.25,14.30,13.95,15.73,15.41,16.99,14.81,15.48,14.85,12.60,8.32,3.36,0.46,1.07,0.30,0.29,0.64,0.23,0.11,0.06,0.63,1.14,3.18,3.17,3.12,3.09,3.03,3.36,3.03,2.74,2.63,10.87,10.49,10.38,...,0.25,0.00,0.00,0.00,0.00,7.34,7.56,7.70,9.50,9.87,12.72,13.99,22.74,22.70,30.36,33.76,41.57,44.08,49.38,52.70,52.8,55.08,58.53,59.03,56.72,52.21,44.03,37.23,27.66,28.66,29.52,20.81,8.71,0.30,0.00,5.69,8.04,10.74,12.67,12.57
3,CDEC:BCB,4.30,4.42,4.62,4.53,4.67,4.90,4.90,5.06,5.11,5.23,5.40,5.43,5.40,5.50,5.65,5.66,5.80,5.81,5.73,5.62,5.44,5.46,5.40,5.78,5.67,5.75,10.49,13.04,13.10,12.82,13.83,14.29,14.02,14.23,13.81,5.18,9.33,9.55,9.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CDEC:BCH,2.88,3.00,3.48,3.84,3.96,4.44,5.40,5.16,3.60,1.80,0.72,0.60,0.12,0.12,0.00,0.12,0.12,0.24,0.00,0.12,0.00,0.00,0.12,0.12,0.00,0.00,0.00,0.00,0.96,0.36,0.36,0.36,0.24,0.12,0.12,0.48,1.44,1.32,0.36,...,0.60,0.48,0.48,0.36,0.60,3.12,3.60,3.48,3.48,3.00,4.80,4.92,9.00,8.76,11.88,15.36,17.88,18.96,18.96,22.92,22.2,21.24,17.04,10.92,2.76,0.60,0.36,0.24,0.48,0.72,0.84,0.60,0.36,0.36,0.24,2.88,4.56,4.68,5.04,6.00


In [8]:
print(ground_measures_test.shape)
ground_measures_test.head()

(700, 58)


,Unnamed: 0,2020-01-07,2020-01-14,2020-01-21,2020-01-28,2020-02-04,2020-02-11,2020-02-18,2020-02-25,2020-03-03,2020-03-10,2020-03-17,2020-03-24,2020-03-31,2020-04-07,2020-04-14,2020-04-21,2020-04-28,2020-05-05,2020-05-12,2020-05-19,2020-05-26,2020-06-02,2020-06-09,2020-06-16,2020-06-23,2020-06-30,2020-12-01,2020-12-08,2020-12-15,2020-12-22,2020-12-29,2021-01-05,2021-01-12,2021-01-19,2021-01-26,2021-02-02,2021-02-09,2021-02-16,2021-02-23,2021-03-02,2021-03-09,2021-03-16,2021-03-23,2021-03-30,2021-04-06,2021-04-13,2021-04-20,2021-04-27,2021-05-04,2021-05-11,2021-05-18,2021-05-25,2021-06-01,2021-06-08,2021-06-15,2021-06-22,2021-06-29
0,CDEC:ADM,4.50,5.50,7.30,8.30,8.10,8.20,9.30,8.50,7.90,7.30,6.20,6.00,7.00,7.90,4.50,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.20,2.10,2.70,3.50,4.50,6.00,6.50,6.20,6.70,7.50,8.50,10.40,12.20,12.30,13.10,14.10,13.90,11.70,8.00,2.40,NaN,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CDEC:AGP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CDEC:ALP,12.72,13.78,17.12,18.07,18.17,18.38,17.71,16.05,14.62,12.79,16.30,17.52,19.35,22.31,21.04,18.58,12.68,2.83,0.14,1.20,0.23,0.20,0.00,0.00,0.01,0.00,4.53,4.35,5.83,7.66,8.98,9.65,10.20,8.43,9.93,17.68,17.33,19.56,20.31,19.90,20.07,22.11,23.81,23.23,18.13,12.40,8.01,3.34,0.31,0.02,0.00,0.13,0.01,0.00,0.00,0.00,0.00
3,CDEC:BCB,12.20,12.20,13.30,13.35,12.85,12.72,12.72,12.80,13.16,12.82,14.33,14.88,16.00,19.29,19.14,20.10,18.84,14.36,9.32,6.69,3.13,1.21,1.22,1.32,1.41,1.39,6.96,7.06,8.11,8.62,9.65,9.36,9.13,9.33,9.78,17.56,17.47,19.12,19.11,19.01,18.96,20.61,21.95,20.14,19.08,17.94,16.82,15.44,11.94,5.91,1.10,0.86,1.04,1.12,1.04,1.17,1.25
4,CDEC:BCH,6.60,5.76,5.16,7.68,4.68,1.32,0.84,0.84,0.24,0.24,0.84,1.20,0.60,0.84,0.84,0.72,0.36,0.24,0.12,0.48,0.36,0.36,0.12,0.24,0.36,0.12,0.12,0.24,0.24,2.40,0.84,1.32,1.44,0.48,0.84,2.52,2.40,2.88,1.56,0.36,0.12,0.96,0.72,0.24,0.24,0.12,0.24,0.12,0.24,0.24,0.24,0.12,0.24,0.12,0.24,0.24,0.36


In [7]:
ground_measures_metadata.head()

,station_id,name,elevation_m,latitude,longitude,state
0,CDEC:ADM,Adin Mountain,1889.76,41.237000,-120.792000,California
1,CDEC:AGP,Agnew Pass,2880.36,37.726631,-119.141731,California
2,CDEC:ALP,Alpha (Smud),2316.48,38.804192,-120.215652,California
3,CDEC:BCB,Blackcap Basin,3139.44,37.066685,-118.773010,California
4,CDEC:BCH,Beach Meadows,2331.72,36.126095,-118.293457,California


In [9]:
print(train_labels.shape)
train_labels.head()

(10878, 268)


,cell_id,2013-01-01,2013-01-08,2013-01-15,2013-01-22,2013-01-29,2013-02-05,2013-02-12,2013-02-19,2013-02-26,2013-03-05,2013-03-12,2013-03-19,2013-03-26,2013-04-02,2013-04-03,2013-04-09,2013-04-16,2013-04-23,2013-04-29,2013-04-30,2013-05-03,2013-05-07,2013-05-14,2013-05-21,2013-05-25,2013-05-28,2013-06-01,2013-06-04,2013-06-08,2013-06-11,2013-06-18,2013-06-25,2013-12-03,2013-12-10,2013-12-17,2013-12-24,2013-12-31,2014-01-07,2014-01-14,...,2019-03-24,2019-03-25,2019-03-26,2019-03-29,2019-04-02,2019-04-07,2019-04-08,2019-04-09,2019-04-16,2019-04-17,2019-04-18,2019-04-19,2019-04-21,2019-04-23,2019-04-27,2019-04-28,2019-04-30,2019-05-01,2019-05-02,2019-05-03,2019-05-07,2019-05-14,2019-05-21,2019-05-28,2019-06-04,2019-06-05,2019-06-08,2019-06-09,2019-06-10,2019-06-11,2019-06-13,2019-06-14,2019-06-18,2019-06-24,2019-06-25,2019-12-03,2019-12-10,2019-12-17,2019-12-24,2019-12-31
0,0003f387-71c4-48f6-b2b0-d853bd4f0aba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.6,NaN,NaN,NaN,NaN,52.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,48.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000617d8-8c14-43e2-b708-7e3a69fe3cc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000ba8d9-d6d5-48da-84a2-1fa54951fae1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,11.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0017d1c4-64cb-426d-9158-3f6521d2dd22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0020c632-3d5c-4509-b4ee-6b63a89bf2ff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
f = open('/content/drive/MyDrive/snocast/grid_cells.geojson')
grid_cells = json.load(f)

In [11]:
grid_cells.keys()

dict_keys(['type', 'crs', 'features'])

In [12]:
len(grid_cells['features'])

18130

In [39]:
grid_features = defaultdict(dict)
for grid_cell in grid_cells['features']:
  cell_id = grid_cell['properties']['cell_id']
  coordinates = grid_cell['geometry']['coordinates']
  region = grid_cell['properties']['region']
  grid_features[cell_id] = {'coordinates': coordinates,
                            'region': region}

for cell_id in train_labels['cell_id'].values:
  grid_features[cell_id]['dataset'] = 'train'

for cell_id in submission_format['cell_id'].values:
  grid_features[cell_id]['dataset'] = 'test'


In [45]:
grid_features_train = defaultdict(dict)
grid_features_test = defaultdict(dict)
for cell_id in grid_features:
  if grid_features[cell_id]['dataset'] == 'test':
    grid_features_test[cell_id] = grid_features[cell_id]
  else:
    grid_features_train[cell_id] = grid_features[cell_id]
print("test count: ", len(grid_features_test))
print("train count: ", len(grid_features_train))

test count:  9066
train count:  9064


In [42]:
grid_features

defaultdict(dict,
            {'0003f387-71c4-48f6-b2b0-d853bd4f0aba': {'coordinates': [[[-118.7189531728198,
                 37.07419231005138],
                [-118.7189531728198, 37.08135922968016],
                [-118.727936325661, 37.08135922968016],
                [-118.727936325661, 37.07419231005138],
                [-118.7189531728198, 37.07419231005138]]],
              'dataset': 'train',
              'region': 'sierras'},
             '000617d8-8c14-43e2-b708-7e3a69fe3cc3': {'coordinates': [[[-107.0767870906308,
                 37.78042352249651],
                [-107.0767870906308, 37.78752314553562],
                [-107.085770243472, 37.78752314553562],
                [-107.085770243472, 37.78042352249651],
                [-107.0767870906308, 37.78042352249651]]],
              'dataset': 'train',
              'region': 'central rockies'},
             '000863e7-21e6-477d-b799-f5675c348627': {'coordinates': [[[-119.4016727887506,
                 37.02400490

## Create baseline submission that uses global monthly mean SWE

In [29]:
mean_swe_by_date = train_labels.mean()

In [23]:
monthly_mean_swe_list = defaultdict(list)

for date, mean_swe in mean_swe_by_date.iteritems():
  monthly_mean_swe_list[date.split('-')[1]].append(mean_swe)

In [24]:
monthly_mean_swe = defaultdict()

for month, mean_swe_list in monthly_mean_swe_list.items():
  monthly_mean_swe[month] = np.mean(mean_swe_list)

In [25]:
monthly_mean_swe

defaultdict(None,
            {'01': 9.822257303250963,
             '02': 12.018673849199772,
             '03': 16.427515790355358,
             '04': 15.24879232790664,
             '05': 9.323579055945377,
             '06': 3.7841599820355665,
             '12': 4.4755022116347725})

In [38]:
submission = submission_format.copy()

In [41]:
for col in submission.columns:
  if col != 'cell_id':
    submission[col] = monthly_mean_swe[col.split('-')[1]]

In [45]:
submission.to_csv('/content/drive/MyDrive/snocast/submission_baseline.csv', index=False)